In [59]:
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
sc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [70]:
df=spark.read.csv('s3://trabajo1-raw/datasets/Water Quality/Cities1/Cities1.csv',inferSchema=True,header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[City: string, Region: string, Country: string, AirQuality: double, WaterPollution: double]

In [53]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- City: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- AirQuality: double (nullable = true)
 |-- WaterPollution: double (nullable = true)

In [54]:
df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------------------+--------------------+-----------+--------------+
|            City|              Region|             Country| AirQuality|WaterPollution|
+----------------+--------------------+--------------------+-----------+--------------+
|   New York City|            New York|United States of ...|46.81603774|    49.5049505|
|Washington, D.C.|District of Columbia|United States of ...|66.12903226|   49.10714286|
|   San Francisco|          California|United States of ...|60.51401869|          43.0|
|          Berlin|                null|             Germany|62.36413043|   28.61271676|
|     Los Angeles|          California|United States of ...|36.62162162|   61.29943503|
+----------------+--------------------+--------------------+-----------+--------------+
only showing top 5 rows

In [55]:
df.describe().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+------------------+-----------+-----------------+------------------+
|summary|  City|            Region|    Country|       AirQuality|    WaterPollution|
+-------+------+------------------+-----------+-----------------+------------------+
|  count|  3963|              3538|       3963|             3963|              3963|
|   mean|   NaN|               NaN|       null| 62.2534517395642| 44.63537214186169|
| stddev|  null|              null|       null|30.94475340271576|25.663910130285892|
|    min|Aachen|    ???il Province|Afghanistan|              0.0|               0.0|
|    max|Zywiec|canton of Fribourg|   Zimbabwe|            100.0|             100.0|
+-------+------+------------------+-----------+-----------------+------------------+

In [64]:
from pyspark.sql.functions import udf

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
## Creando una columna con datos 1 = mayor que la media y 0 = menor que la media

AQMean = udf(lambda AirQuality: "0" if AirQuality <= 62.25 else "1")

df.withColumn("AQ>Mean", AQMean(df.AirQuality)).show(10,False)



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------------------+------------------------+-----------+--------------+-------+
|City            |Region              |Country                 |AirQuality |WaterPollution|AQ>Mean|
+----------------+--------------------+------------------------+-----------+--------------+-------+
|New York City   |New York            |United States of America|46.81603774|49.5049505    |0      |
|Washington, D.C.|District of Columbia|United States of America|66.12903226|49.10714286   |1      |
|San Francisco   |California          |United States of America|60.51401869|43.0          |0      |
|Berlin          |null                |Germany                 |62.36413043|28.61271676   |1      |
|Los Angeles     |California          |United States of America|36.62162162|61.29943503   |0      |
|Bern            |Canton of Bern      |Switzerland             |94.31818182|12.5          |1      |
|Geneva          |Canton of Geneva    |Switzerland             |71.53846154|17.37288136   |1      |


In [72]:
## No se guarda la columna AQMean

df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------------------+--------------------+-----------+--------------+
|            City|              Region|             Country| AirQuality|WaterPollution|
+----------------+--------------------+--------------------+-----------+--------------+
|   New York City|            New York|United States of ...|46.81603774|    49.5049505|
|Washington, D.C.|District of Columbia|United States of ...|66.12903226|   49.10714286|
|   San Francisco|          California|United States of ...|60.51401869|          43.0|
|          Berlin|                null|             Germany|62.36413043|   28.61271676|
|     Los Angeles|          California|United States of ...|36.62162162|   61.29943503|
+----------------+--------------------+--------------------+-----------+--------------+
only showing top 5 rows

In [12]:
## Ciudades de Colombia que tienen contaminanción del agua superior a la media mundial

df.filter(df['Country']=='Colombia').filter(df['WaterPollution'] >44.63).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+--------------------+--------+-----------+--------------+
|             City|              Region| Country| AirQuality|WaterPollution|
+-----------------+--------------------+--------+-----------+--------------+
|           Bogota|                null|Colombia|23.35025381|   48.15789474|
|          Pereira|Risaralda Department|Colombia|       25.0|          75.0|
|     Barranquilla|Atlantico Department|Colombia|52.77777778|          50.0|
|      Santa Marta|Magdalena Department|Colombia|       50.0|   71.42857143|
|           Ibague|   Tolima Department|Colombia|56.61764706|   79.04411765|
|            Tunja|   Boyaca Department|Colombia|      100.0|          75.0|
|San Juan de Pasto|   Narino Department|Colombia|       70.0|          75.0|
|       Valledupar|    Cesar Department|Colombia|       50.0|          50.0|
|            Neiva|    Huila Department|Colombia|       75.0|          75.0|
|        Cartagena|  Bolivar Department|Colombia|       60.0|          65.0|

In [13]:
## Regiones con más ciudades en dataframe

df.groupBy('Country','Region').count().orderBy('count',ascending=False).show(20,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+----------------------+-----+
|Country                 |Region                |count|
+------------------------+----------------------+-----+
|United Kingdom          |England               |142  |
|United States of America|California            |122  |
|Czech Republic          |null                  |57   |
|United States of America|Texas                 |51   |
|United States of America|Florida               |48   |
|Canada                  |Ontario               |47   |
|France                  |null                  |42   |
|Canada                  |British Columbia      |38   |
|Philippines             |null                  |36   |
|Brazil                  |Sao Paulo             |32   |
|United States of America|Georgia               |31   |
|United States of America|New York              |31   |
|Germany                 |North Rhine-Westphalia|30   |
|United States of America|Washington            |29   |
|United States of America|North Carolina        

In [14]:
from pyspark.sql.types import StringType,DoubleType,IntegerType


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df.filter(df['Country']=='Colombia').filter(df['AirQuality'] >30).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+--------------------+--------+-----------+--------------+
|             City|              Region| Country| AirQuality|WaterPollution|
+-----------------+--------------------+--------+-----------+--------------+
|             Cali|Valle del Cauca D...|Colombia|67.04545455|       42.1875|
|     Barranquilla|Atlantico Department|Colombia|52.77777778|          50.0|
|      Santa Marta|Magdalena Department|Colombia|       50.0|   71.42857143|
|           Ibague|   Tolima Department|Colombia|56.61764706|   79.04411765|
|            Tunja|   Boyaca Department|Colombia|      100.0|          75.0|
|      Bucaramanga|Santander Department|Colombia|       40.0|          25.0|
|San Juan de Pasto|   Narino Department|Colombia|       70.0|          75.0|
|          Armenia|  Quindio Department|Colombia|       62.5|          37.5|
|          Popayan|    Cauca Department|Colombia|      100.0|          25.0|
|       Valledupar|    Cesar Department|Colombia|       50.0|          50.0|

In [17]:
from pyspark.sql.functions import pandas_udf, PandasUDFType

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
def critical_airq(AirQuality):
    quality_left= 2 * df.mean()-AirQuality

    return quality_left

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
## Intentamos guardar el DataFrame con la nueva columna en una nueva variable, pero al correr el código nos mostró el siguiente error 

Rdf.write.format('csv').option('header','true').save('s3://trabajo1-refined/datarefine/water_quality/Rdf_csv',mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'NoneType' object has no attribute 'write'
Traceback (most recent call last):
AttributeError: 'NoneType' object has no attribute 'write'



In [74]:
df.write.format('csv').option('header','true').save('s3://trabajo1-refined/datarefine/water_quality/df_csv',mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…